<a href="https://colab.research.google.com/github/zubayerhimel/test/blob/master/Implementing_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot
from sklearn.model_selection import train_test_split

In [ ]:
# load a single file as a numpy array
def load_file(filepath):
  dataframe = read_csv(filepath)
  return dataframe.values

In [ ]:
# importing file from drive
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
def dataPreprocessing(filePath, repeats):
  data = load_file(filePath)
  X, y = data[:, :-1], data[:, -1]
 
  # split into train test sets
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

  # Modify Train X 
  trainData = list()
  trainData.append(X_train)
  X_train = dstack(trainData)

  # Modify Train y
  y_train = np.reshape(y_train, (-1, 1))
  y_train = y_train - 1
  y_train = to_categorical(y_train);

  # Modify Test X 
  testData = list()
  testData.append(X_test)
  X_test = dstack(testData)

  # Modify Test y
  y_test = np.reshape(y_test, (-1, 1))
  y_test = y_test - 1
  y_test = to_categorical(y_test);

  # repeat experiment
  scores = list()
  for r in range(repeats):
    # score = vanilla_LSTM(X_train, y_train, X_test, y_test)
    score = stack_LSTM(X_train, y_train, X_test, y_test)
    score = score * 100.0
    print('Repeat number => %d: %.3f' % (r+1, score))
    scores.append(score)
	# summarize results
  summarize_results(scores)

In [ ]:
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [ ]:
# fit and evaluate a vanilla lstm model
def vanilla_LSTM(X_train, y_train, X_test, y_test):
  verbose, epochs, batch_size = 0, 15, 64
  n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
  model = Sequential()

  model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
  model.add(Dropout(0.5))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(n_outputs, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
  _, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
  
  return accuracy

In [ ]:
# fit and evaluate stack lstm
def stack_LSTM(X_train, y_train, X_test, y_test):
  verbose, epochs, batch_size = 0, 15, 64
  n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
  model = Sequential()

  model.add(LSTM(300, activation='relu', return_sequences=True, input_shape=(n_timesteps,n_features)))
  model.add(LSTM(200, activation='relu', return_sequences=True))
  model.add(LSTM(100, activation='relu', return_sequences=True))
  model.add(LSTM(50, activation='relu', return_sequences=True))
  model.add(LSTM(25, activation='relu'))
  model.add(Dense(30, activation='relu'))
  model.add(Dense(20, activation='relu'))
  model.add(Dense(10, activation='relu'))
  # model.add(Dropout(0.5))

  model.add(Dense(n_outputs, activation='sigmoid'))
  model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
  model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
  _, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
  
  return accuracy

In [ ]:
datasetList = list()
datasetList.append('/content/gdrive/My Drive/Colab-Notebooks/dataset/merged.csv')
datasetList.append('/content/gdrive/My Drive/Colab-Notebooks/dataset/merged2.csv')

In [ ]:
for i in datasetList:
  dataPreprocessing(i, 2)

Repeat number => 1: 93.982
Repeat number => 2: 93.859
Repeat number => 3: 93.954
Repeat number => 4: 94.075
Repeat number => 5: 93.955
[93.98240447044373, 93.85884404182434, 93.95391941070557, 94.07467246055603, 93.95512342453003]
Accuracy: 93.965% (+/-0.069)
Repeat number => 1: 93.563
Repeat number => 2: 93.240
Repeat number => 3: 93.562
Repeat number => 4: 93.263
Repeat number => 5: 93.209
[93.56271028518677, 93.24023723602295, 93.56199502944946, 93.26311945915222, 93.2087779045105]
Accuracy: 93.367% (+/-0.160)
